In [1]:
import numpy as np
import sys
# option to import from github folder
# sys.path.insert(0, 'C:/Users/carse/github/suite2p')
import suite2p
from suite2p.run_s2p import run_s2p
import os 
import glob
import admin_functions1 as adfn
from skimage import io
from skimage.external.tifffile import imread, TiffFile 
from PIL import Image
# Jupyter specific autoreloading for external functions (in case changes are made)
%load_ext autoreload
%autoreload
# %matplotlib widget

# Define paths
#---------------------------------------------------------------------------
Fdrop = '/Users/dominicburrows/Dropbox/PhD/analysis/'
Fmac = '/Users/dominicburrows/Documents/'
F10t = '/Volumes/Dominic 10tb/'
Ffig = '/Users/dominicburrows/Dropbox/PhD/figures/'
Ftm = '/Volumes/Dominic Time Machine/'

## Preprocess files

In [ ]:
#PUT ALL TIFFS INTO SINGLE FILE TO RUN SUITE2P
numlist = adfn.name_zero('front', 3,3, 1)
for t in numlist:
    
    fish = 'PTZ-WILDTYPE-' + t + '/2photon/sess-01-6dpf/'
    read_path = Ftm + 'Project/' + fish
    save_path = F10t + 'Data2process/Project/' + fish[:fish.find('2photon')-4]+ os.sep + fish[:fish.find('2photon')-1]
    os.mkdir(save_path)
        
    for y in range(10):
        os.chdir(read_path)
        blnlist = sorted(glob.glob('*BLN' + '*plane' + str(y) + '*tif'))
        p5list = sorted(glob.glob('*PTZ05' + '*plane' + str(y) + '*tif'))
        p20list = sorted(glob.glob('*PTZ20' + '*plane' + str(y) + '*tif'))
        alllist = np.append(blnlist, np.append(p5list, p20list))
        pid = alllist[0][alllist[0].find('PTZ-WILDTYPE'):alllist[0].find('WILDTYPE')+11] + '_' + alllist[0][alllist[0].find('pla'):alllist[0].find('pla')+6 ]
        os.chdir(save_path)
        os.mkdir(pid)

        for x in range(len(alllist)):
            io.imsave(arr = imread(read_path + alllist[x]),  fname= save_path + '/' + pid + '/' + str(alllist[x]))


In [ ]:
#SAVE REGISTERED TIFFS IN TEMPORARY FOLDER FOR SUITE2P
#loop through registered tiffs and save new folder containing them - to run suite2p
#------------------------------------------------------------------------------------------------------

# Define which fish 
#---------------------------------------------------------------------------
for t in range(3,4):
    
    fish = 'PTZ-WILDTYPE-0' + str(t) + '/2photon/sess-01-6dpf/'
    finalpath = Ftm + 'Project/' + fish
    tmpath = Ftm + 'Project/' + fish[:fish.find('2photon')-1]

    for i in range(3):
        if i == 0:
            condition = 'BLN'
        elif i ==1:
            condition = 'PTZ05'
        elif i ==2:
            condition = 'PTZ20'
        
        for y in range(10):
            os.chdir(finalpath)
            alllist = sorted(glob.glob('*' + condition + '*plane' + str(y) + '*'))
            pid = alllist[0][alllist[0].find('PTZ-WILDTYPE'):alllist[0].find('WILDTYPE')+11] + alllist[0][alllist[0].find('dpf')+3:alllist[0].find('dpf')+8] + alllist[0][alllist[0].find('pla'):alllist[0].find('pla')+6 ]
            os.chdir(Ftm + 'Project/' + fish[:16])
            os.mkdir(pid)

            for x in range(len(alllist)):
                io.imsave(arr = imread(finalpath + alllist[x]),  fname= tmpath + '/' + pid + '/' + str(alllist[x]))

In [ ]:
#REGISTRATION FAILURE on single plane - EXTRACT RAW TIFF FOR PLANE - suite2p without registration
# Loop through raw tiffs to pull out individual plane 
#--------------------------------------------------------------------------------------

# Define which fish 
#---------------------------------------------------------------------------
experiment = 'PTZ-WILDTYPE'
date = '190711-WILDTYPE'
fish = 'F12-PTZ05-WILDTYPE-2photon-6dpf-gc6s-15%-sess-01-run-01'
finalpath = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish
os.chdir(finalpath)
plane = 0

planelist = sorted(glob.glob('*' + str(plane) + '.tif'))
newfold = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish[:18] + 'plane' + str(plane)
os.mkdir(newfold)

for i in range(len(planelist)):
    io.imsave(arr = imread(finalpath + os.sep + planelist[i]),  fname= newfold + os.sep + str(planelist[i]))

## Define suite2p settings

In [3]:
# set your options for running
# overwrites the run_s2p.default_ops
#-----------------------------------------

ops = {
            'fast_disk': [], # used to store temporary binary file, defaults to save_path0 (set as a string NOT a list)
            'save_path0': [], # stores results, defaults to first item in data_path
            'delete_bin': 1, # whether to delete binary file after processing
            # main settings
            'nplanes' : 1, # each tiff has these many planes in sequence
            'nchannels' : 1, # each tiff has these many channels per plane
            'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
            'diameter':5, # def = 5, this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
            'tau':  1., # this is the main parameter for deconvolution
            'fs': 2.73,  # sampling rate (total across planes)
            # output settings
            'save_mat': False, # whether to save output as matlab files
            'combined': False, # combine multiple planes into a single result /single canvas for GUI
            # parallel settings
            'num_workers': 0, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
            'num_workers_roi': 2, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
            # registration settings
            'do_registration': True, # whether to register data
            'nimg_init': 200, #def = 200, subsampled frames for finding reference image
            'batch_size': 200, # number of frames per batch
            'maxregshift': 0.05, # max allowed registration shift, as a fraction of frame max(width and height)
            'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
            'reg_tif': 1, # whether to save registered tiffs
            'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
            #'nonrigid' : 1,
            #'maxregshiftNR': 0.05,
            # cell detection settings
            'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
            'navg_frames_svd': 5000, # max number of binned frames for the SVD
            'nsvd_for_roi': 1000, # max number of SVD components to keep for ROI detection
            'max_iterations': 20, # maximum number of iterations to do cell detection
            'ratio_neuropil': 6.0, # ratio between neuropil basis size and cell radius
            'ratio_neuropil_to_cell': 3, # minimum ratio between neuropil radius and cell radius
            'tile_factor': 1, # use finer (>1) or coarser (<1) tiles for neuropil estimation during cell detection
            'threshold_scaling': 1., # adjust the automatically determined threshold by this scalar multiplier
            'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
            'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
            'outer_neuropil_radius': np.inf, # maximum neuropil radius
            'min_neuropil_pixels':350, # minimum number of pixels in the neuropil
            # deconvolution settings
            'baseline': 'maximin', # baselining mode
            'win_baseline': 60., # window for maximin
            'sig_baseline': 10., # smoothing constant for gaussian filter
            'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
            'neucoeff': .7,  # neuropil coefficient
          }
        

## Run suite2p

In [ ]:
# provide an h5 path in 'h5py' or a tiff path in 'data_path'
# db overwrites any ops (allows for experiment specific settings)
# db = {
#       'h5py': [], # a single h5 file path
#       'h5py_key': 'data',
#       'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
#       'data_path': ['C:/Users/carse/tiffs'], # a list of folders with tiffs 
#                                              # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
#       'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
#       'fast_disk': 'C:/BIN', # string which specifies where the binary file will be stored (should be an SSD)
#     }

# run one experiment
# opsEnd=run_s2p(ops=ops,db=db)

In [4]:
# make a list of databases and loop over them
#---------------------------------------------
db = []

numlist = adfn.name_zero('front', 3,3, 1)
for i in numlist:
    fish = 'PTZ-WILDTYPE-' + i
    read_path = F10t + 'Data2process/Project/PTZ-WILDTYPE/' + fish + '/'
    for y in range(10):
        plane = 'PTZ-WILDTYPE-' + str(i) +  '_' + 'plane' + str(y)
        final_path = read_path + plane
        os.chdir(final_path)
        db.append({'data_path': [final_path]})

In [5]:
db

[{'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane0']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane1']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane2']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane3']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane4']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane5']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane6']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane7']},
 {'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE

In [6]:
#RUN
#-----------------
#Fme =  F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish[:18] + 'plane' + str(plane)
#db = []
#db.append({'data_path': [Fme]})
                        
for dbi in db:
    opsEnd=run_s2p(ops=ops,db=dbi)

{'data_path': ['/Volumes/Dominic 10tb/Data2process/Project/PTZ-WILDTYPE/PTZ-WILDTYPE-03/PTZ-WILDTYPE-03_plane0']}
tif
** Found 150 tifs - converting to binary **
800 frames of binary, time 3.40 sec.
1600 frames of binary, time 9.83 sec.
2400 frames of binary, time 19.18 sec.
3200 frames of binary, time 25.43 sec.
4000 frames of binary, time 28.75 sec.
4800 frames of binary, time 32.46 sec.
5600 frames of binary, time 35.77 sec.
6400 frames of binary, time 39.06 sec.
7200 frames of binary, time 44.41 sec.
8000 frames of binary, time 53.47 sec.
8800 frames of binary, time 63.49 sec.
9600 frames of binary, time 72.95 sec.
time 314.22 sec. Wrote 29484 tiff frames to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 29484 frames
Reference frame, 6.23 sec.


/Users/dominicburrows/anaconda3/lib/python3.7/site-packages/suite2p/registration/nonrigid.py:197: RuntimeWarning: divide by zero encountered in true_divide
  snr = X1max / Xmax # computes snr


1000/29484 frames, 27.70 sec.
2000/29484 frames, 55.65 sec.
3000/29484 frames, 83.20 sec.
4000/29484 frames, 111.87 sec.
5000/29484 frames, 139.63 sec.
6000/29484 frames, 167.32 sec.
7000/29484 frames, 196.82 sec.
8000/29484 frames, 225.53 sec.
9000/29484 frames, 253.75 sec.
10000/29484 frames, 283.09 sec.
11000/29484 frames, 312.62 sec.
12000/29484 frames, 341.37 sec.
13000/29484 frames, 369.53 sec.
14000/29484 frames, 396.81 sec.
15000/29484 frames, 424.30 sec.
16000/29484 frames, 453.80 sec.
17000/29484 frames, 484.41 sec.
18000/29484 frames, 513.05 sec.
19000/29484 frames, 541.03 sec.
20000/29484 frames, 569.11 sec.
21000/29484 frames, 597.95 sec.
22000/29484 frames, 627.59 sec.
23000/29484 frames, 659.91 sec.
24000/29484 frames, 694.09 sec.
25000/29484 frames, 725.66 sec.
26000/29484 frames, 755.14 sec.
27000/29484 frames, 784.16 sec.
28000/29484 frames, 814.27 sec.
29000/29484 frames, 842.95 sec.
29484/29484 frames, 856.62 sec.
bad frames file path: /Volumes/Dominic 10tb/Data2pro

/Users/dominicburrows/anaconda3/lib/python3.7/site-packages/suite2p/registration/nonrigid.py:197: RuntimeWarning: divide by zero encountered in true_divide
  snr = X1max / Xmax # computes snr


1000/29484 frames, 29.11 sec.
2000/29484 frames, 58.14 sec.
3000/29484 frames, 87.69 sec.
4000/29484 frames, 118.52 sec.
5000/29484 frames, 148.12 sec.
6000/29484 frames, 177.60 sec.
7000/29484 frames, 207.08 sec.
8000/29484 frames, 235.56 sec.
9000/29484 frames, 264.39 sec.
10000/29484 frames, 293.75 sec.
11000/29484 frames, 323.16 sec.
12000/29484 frames, 352.57 sec.
13000/29484 frames, 382.37 sec.
14000/29484 frames, 411.71 sec.
15000/29484 frames, 442.45 sec.
16000/29484 frames, 472.27 sec.
17000/29484 frames, 502.65 sec.
18000/29484 frames, 532.49 sec.
19000/29484 frames, 563.00 sec.
20000/29484 frames, 592.62 sec.
21000/29484 frames, 623.10 sec.
22000/29484 frames, 652.96 sec.
23000/29484 frames, 682.23 sec.
24000/29484 frames, 710.35 sec.
25000/29484 frames, 739.08 sec.
26000/29484 frames, 767.93 sec.
27000/29484 frames, 796.82 sec.
28000/29484 frames, 826.84 sec.
29000/29484 frames, 856.97 sec.
29484/29484 frames, 871.16 sec.
bad frames file path: /Volumes/Dominic 10tb/Data2pro

...

1000/29484 frames, 28.85 sec.
2000/29484 frames, 57.96 sec.
3000/29484 frames, 87.94 sec.
4000/29484 frames, 117.76 sec.
5000/29484 frames, 147.35 sec.
6000/29484 frames, 177.02 sec.
7000/29484 frames, 207.54 sec.
8000/29484 frames, 237.47 sec.
9000/29484 frames, 267.35 sec.
10000/29484 frames, 299.54 sec.
11000/29484 frames, 328.96 sec.
12000/29484 frames, 358.01 sec.
13000/29484 frames, 387.57 sec.
14000/29484 frames, 416.82 sec.
15000/29484 frames, 447.44 sec.
16000/29484 frames, 477.68 sec.
17000/29484 frames, 507.18 sec.
18000/29484 frames, 537.09 sec.
19000/29484 frames, 567.03 sec.
20000/29484 frames, 596.87 sec.
21000/29484 frames, 627.75 sec.
22000/29484 frames, 657.37 sec.
23000/29484 frames, 687.64 sec.
24000/29484 frames, 717.01 sec.
25000/29484 frames, 746.83 sec.
26000/29484 frames, 776.95 sec.
27000/29484 frames, 807.59 sec.
28000/29484 frames, 837.40 sec.
29000/29484 frames, 867.19 sec.
29484/29484 frames, 880.92 sec.
bad frames file path: /Volumes/Dominic 10tb/Data2pro

/Users/dominicburrows/anaconda3/lib/python3.7/site-packages/suite2p/registration/nonrigid.py:197: RuntimeWarning: divide by zero encountered in true_divide
  snr = X1max / Xmax # computes snr


1000/29484 frames, 29.53 sec.
2000/29484 frames, 58.19 sec.
3000/29484 frames, 87.46 sec.
4000/29484 frames, 116.52 sec.
5000/29484 frames, 145.83 sec.
6000/29484 frames, 176.00 sec.
7000/29484 frames, 206.39 sec.
8000/29484 frames, 236.36 sec.
9000/29484 frames, 266.90 sec.
10000/29484 frames, 296.38 sec.
11000/29484 frames, 326.97 sec.
12000/29484 frames, 356.70 sec.
13000/29484 frames, 386.16 sec.
14000/29484 frames, 416.50 sec.
15000/29484 frames, 446.05 sec.
16000/29484 frames, 476.34 sec.
17000/29484 frames, 506.46 sec.
18000/29484 frames, 536.56 sec.
19000/29484 frames, 566.53 sec.
20000/29484 frames, 596.76 sec.
21000/29484 frames, 626.84 sec.
22000/29484 frames, 656.49 sec.
23000/29484 frames, 686.84 sec.
24000/29484 frames, 716.64 sec.
25000/29484 frames, 745.67 sec.
26000/29484 frames, 774.52 sec.
27000/29484 frames, 803.62 sec.
28000/29484 frames, 832.93 sec.
29000/29484 frames, 862.29 sec.
29484/29484 frames, 876.35 sec.
bad frames file path: /Volumes/Dominic 10tb/Data2pro

/Users/dominicburrows/anaconda3/lib/python3.7/site-packages/suite2p/registration/nonrigid.py:197: RuntimeWarning: divide by zero encountered in true_divide
  snr = X1max / Xmax # computes snr


1000/29484 frames, 28.98 sec.
2000/29484 frames, 58.40 sec.
3000/29484 frames, 88.72 sec.
4000/29484 frames, 118.04 sec.
5000/29484 frames, 148.01 sec.
6000/29484 frames, 178.27 sec.
7000/29484 frames, 208.04 sec.
8000/29484 frames, 238.02 sec.
9000/29484 frames, 268.05 sec.
10000/29484 frames, 298.05 sec.
11000/29484 frames, 328.38 sec.
12000/29484 frames, 358.40 sec.
13000/29484 frames, 388.30 sec.
14000/29484 frames, 418.23 sec.
15000/29484 frames, 448.28 sec.
16000/29484 frames, 478.15 sec.
17000/29484 frames, 508.31 sec.
18000/29484 frames, 538.65 sec.
19000/29484 frames, 568.14 sec.
20000/29484 frames, 598.50 sec.
21000/29484 frames, 628.79 sec.
22000/29484 frames, 658.60 sec.
23000/29484 frames, 688.13 sec.
24000/29484 frames, 717.96 sec.
25000/29484 frames, 747.95 sec.
26000/29484 frames, 777.51 sec.
27000/29484 frames, 807.56 sec.
28000/29484 frames, 838.05 sec.
29000/29484 frames, 867.52 sec.
29484/29484 frames, 881.48 sec.
bad frames file path: /Volumes/Dominic 10tb/Data2pro

## Backup data

In [ ]:
#BACKUP NONREGISTERED TIFFS IN SUITE2P OUTPUT FORMAT
#After doing suite2p on single plane that failed registration, can back up these tiffs 
#in same format as suite2p output
#-----------------------------------------------------------------------------------
experiment = 'PTZ-WILDTYPE'
date = '190711-WILDTYPE'
fish = 'F12-PTZ05-WILDTYPE-2photon-6dpf-gc6s-15%-sess-01-run-01-suite2p'
finalpath = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish + os.sep + 'plane9/reg_tif/'
fonalpath = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish + os.sep + 'plane9/reg_tif*/'
os.chdir(finalpath)
os.mkdir(fonalpath)
tiflist = sorted(glob.glob('*.tif'))
counter = 0

for i in range(50):
    omlist = []
    
    if i == 49:
        for y in range(28):
            omlist.append(Image.fromarray(io.imread(finalpath + tiflist[counter])))
            counter+=1
            
        omlist[0].save(fonalpath + "file_chan049.tif", save_all=True,
               append_images=omlist[1:])
    
    for y in range(200):
        omlist.append(Image.fromarray(io.imread(finalpath + tiflist[counter])))
        counter+=1
    
    if i < 10:
        omlist[0].save(fonalpath + "file_chan00" + str(i) + ".tif", save_all=True,
               append_images=omlist[1:])
    elif i > 9:
        omlist[0].save(fonalpath + "file_chan0" + str(i) + ".tif", save_all=True,
               append_images=omlist[1:])
        

In [20]:
#BACKUP OPS files from raw suite2p data
#--------------------------------------------------------------------------------------
experiment = 'PTZ-WILDTYPE-06'
savepath = Ftm + 'Project/' + experiment + '/2photon/sess-01-6dpf/'

for i in range(3):
    if i == 0:
        condition = 'BLN_'
        candition = 'BLN'
    elif i ==1:
        condition = 'PTZ0'
        candition = 'PTZ05'
    elif i ==2:
        condition = 'PTZ2'
        candition = 'PTZ20'

               
    for y in range(10):
        saveops = np.load(Ftm + 'Project/' + experiment + os.sep + experiment + '_' + condition + 'plane' + str(y) + os.sep + 'suite2p/plane0/ops.npy')
        savestat = np.load(Ftm + 'Project/' + experiment + os.sep + experiment + '_' + condition + 'plane' + str(y) + os.sep + 'suite2p/plane0/stat.npy')
        np.save(savepath + experiment + '_2photon_sess-01-6dpf_' + candition + '_run-01_plane' + str(y) + '_ops.npy', saveops)
        np.save(savepath + experiment + '_2photon_sess-01-6dpf_' + candition + '_run-01_plane' + str(y) + '_stat.npy', savestat)